In [ ]:
import os
from os import path
import glob
import re

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
from tqdm import tqdm
from scipy.stats import beta, binned_statistic, norm
from scipy.integrate import simps

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini, stellar_radius
from twoface.log import log as logger
logger.setLevel(100)

import mesa_reader as mr
from helpers import tcirc_inv, compute_dlne, MESAHelper

In [ ]:
plot_path = '../paper/figures/'
# table_path = '../data/'
table_path = '../../twoface/paper/1-catalog/tables/'

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'), character_as_bytes=False)
clean_uni = unimodal[(unimodal['clean_flag'] == 0)]
high_logg = clean_uni[clean_uni['LOGG'] > 2]
print('{0} unimodal, {1} clean, {2} high logg'
      .format(len(unimodal), len(clean_uni), len(high_logg)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
ax.scatter(high_logg['TEFF'], high_logg['LOGG'], 
           marker='o', s=20, alpha=0.7, c='k', linewidth=0)
ax.errorbar(high_logg['TEFF'], high_logg['LOGG'],
            xerr=high_logg['TEFF_ERR'], 
            yerr=high_logg['LOGG_ERR'], 
            linestyle='none', zorder=-10, alpha=0.5, c='#aaaaaa')

print('{0} systems total, {1} with M1'.format(len(high_logg), np.isfinite(high_logg['M1']).sum()))

ax.set_xlim(5700, 3800)
ax.set_ylim(4., 1.8)

axes[1].hist(high_logg['M1'][np.isfinite(high_logg['M1'])],
             bins=np.linspace(0.75, 3.25, 11));

axes[0].set_xlabel(r'$T_{\rm eff}$'+' [{0:latex}]'.format(u.Kelvin))
axes[0].set_ylabel(r'$\log g$')
axes[1].set_xlabel(r'$M_1$' + ' [{0:latex}]'.format(u.Msun))
axes[1].set_ylabel('$N$')

fig.tight_layout()
fig.savefig('../paper/figures/logg-teff-m1.pdf')

In [ ]:
# overlapping
# fig, axes = plt.subplots(2, 3, figsize=(12, 8.5), sharex=True, sharey=True)
fig, axes = plt.subplots(2, 3, figsize=(10, 7.08), sharex=True, sharey=True)
x1 = 2.
x2 = 3.8
h = 0.4
grid = np.linspace(x1, x2-h, 6)
for i, (l, r) in enumerate(reversed(list(zip(grid, grid+h)))):
# non-overlapping
# fig, axes = plt.subplots(2, 2, figsize=(7, 7), sharex=True, sharey=True)
# grid = np.linspace(2, 3.8, 4+1)
# for i, (l, r) in enumerate(reversed(list(zip(grid[:-1], grid[1:])))):
    mask = (clean_uni['LOGG'] > l) & (clean_uni['LOGG'] <= r)
    subcat = clean_uni[mask]
    axes.flat[i].scatter(subcat['P'].value, subcat['e'], c='k', 
                         s=style['s'], alpha=0.9)
    axes.flat[i].errorbar(subcat['P'].value, subcat['e'], 
                          xerr=subcat['P_err'].value, yerr=subcat['e_err'],
                          **estyle)
    axes.flat[i].axvline(10., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].axvline(100., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].text(1, 0.95, r'${0:.2f} < \log g \leq {1:.2f}$'.format(l, r), 
                      ha='left', va='top', fontsize=16, backgroundcolor='w')
    
#     P_min = Pcirc(t=0.5*u.Gyr, M1=1.36*u.Msun, logg=r).to(u.day).value
#     P_max = Pcirc(t=4*u.Gyr, M1=1.36*u.Msun, logg=l).to(u.day).value
#     axes.flat[i].axvspan(P_min, P_max, linewidth=0, color='tab:orange', zorder=-100, alpha=0.1)

axes.flat[0].set_xscale('log')

axes.flat[0].xaxis.set_ticks(10**np.arange(0, 3+0.1))
ticks = np.concatenate([i*10**np.arange(-1, 4+0.1) for i in range(1, 10)])
ticks.sort()
axes.flat[0].xaxis.set_ticks(ticks, minor=True)

axes.flat[0].set_xlim(0.8, 1800)
axes.flat[0].set_ylim(-0.05, 1.)

# axes[1, 0].set_xlabel('period, $P$ [day]')
axes[1, 1].set_xlabel('period, $P$ [day]')
axes[0, 0].set_ylabel('eccentricity, $e$')
axes[1, 0].set_ylabel('eccentricity, $e$')

axes[0, 1].set_title(r'${\bf APOGEE\,\,binaries}$', fontsize=23)
fig.tight_layout()

fig.savefig(path.join(plot_path, 'P-e-grid.pdf'))

# P vs. e

In [ ]:
cmap = plt.get_cmap('inferno')

style = dict(marker='o', edgecolor='#555555', linewidth=0.5,
             alpha=0.75, vmin=2, vmax=4, cmap=cmap,
             s=30, c=high_logg['LOGG'], rasterized=True)

estyle = dict(marker='', linestyle='none',
              ecolor='#aaaaaa', alpha=0.7, zorder=-10, elinewidth=1.5)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5.6), sharex=True)

P_surf = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                           high_logg['e'], 0.5*u.Msun)
P_ratio = high_logg['P'] / P_surf

# Plotting
cs = axes[0].scatter(P_ratio, high_logg['e'], zorder=10,
                     **style)
axes[0].errorbar(P_ratio, high_logg['e'],
                 xerr=high_logg['P_err']/P_surf,
                 yerr=high_logg['e_err'], 
                 **estyle)

axes[1].scatter(P_ratio, high_logg['K'], zorder=10,
                **style)
axes[1].errorbar(P_ratio, high_logg['K'].value,
                 xerr=high_logg['P_err']/P_surf,
                 yerr=high_logg['K_err'].value, 
                 **estyle)

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 1)
axes[0].set_xlim(0.7, 1000)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 1+0.2, 0.2))

# ---

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.88, 0.155, 0.025, 0.56])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))
cb.ax.invert_yaxis()

axes[0].axvspan(8., 12, zorder=-5, 
                color='tab:blue', alpha=0.1, linewidth=0)

for ax in axes:
    mn, mx = ax.get_xlim()
    ax2 = ax.twiny()
    ax2.set_xscale('log')
    ax2.set_xlim(mn**(2/3), mx**(2/3))
    ax2.set_xlabel(r'$a\,(1-e) / R_1$', labelpad=15)

fig.tight_layout()
fig.subplots_adjust(top=0.72, right=0.85, wspace=0.4)

fig.suptitle(r'${\bf APOGEE\,\,binaries}$', y=0.94, x=0.46, fontsize=26)

fig.savefig(path.join(plot_path, 'P-e-K.pdf'), dpi=250)

# dlne vs e

In [ ]:
mesa = MESAHelper('../mesa/')

In [ ]:
dlnes = []
for row in tqdm(high_logg):
    if row['LOGG'] < -99:
        dlnes.append(np.nan)
        continue
    
    M1 = 1.36*u.Msun
    M2 = 0.5*u.Msun
    a = np.cbrt(row['P']**2/(2*np.pi)**2 * (G * (M1 + M2))).to(u.au)
    dlne = compute_dlne(row['LOGG'], M1=M1, M2=M2, a=a, 
                        mesa_helper=mesa)
    dlnes.append(dlne)
dlnes = np.array(dlnes)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

c = ax.scatter(-dlnes, high_logg['e'], **style)
ax.errorbar(-dlnes, high_logg['e'], 
            yerr=high_logg['e_err'], 
            **estyle)

# derppp = (-dlnes > 1E1) & (high_logg['e'] > 0.1)
# ax.scatter(-dlnes[derppp], high_logg[derppp]['e'], c='r')

cb = fig.colorbar(c)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))
cb.ax.invert_yaxis()

ax.set_xlim(1E9, 1E-9)
ax.set_ylim(-0.05, 1.)
ax.set_xscale('log')

ax.set_xlabel(r'$-\Delta \ln e$')
ax.set_ylabel('$e$')

fig.savefig(path.join(plot_path, 'dlne.pdf'), dpi=250)